<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/agent_framework/autogpt-with-langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Planning 设计模式示例

In [2]:
!pip install langchain langchain_experimental openai tiktoken google-search-results chromadb -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-functions-tool-retrieval-agent 0.0.1 requires tiktoken<0.6.0,>=0.5.1, but you have tiktoken 0.6.0 which is incompatible.


In [3]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
# from dotenv import load_dotenv
# load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['SERPAPI_API_KEY'] = 'you serp api key'
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

True

In [53]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]


In [54]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [55]:
from langchain.vectorstores import Chroma

vectordb = Chroma(persist_directory="./chroma", embedding_function=embedding)

In [56]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [57]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Xiaoming",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectordb.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True


In [58]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  # agent.run(["write a weather report for Beijing today"])
  agent.run(["Write a report on today's weather in Wuhan, save it as wuhan_weather.txt"])
  # agent.run(["Write a recent news article about new energy vehicles"])
  print(cb)



> Entering new LLMChain chain...


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Prompt after formatting:
System: You are Xiaoming, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Write a report on today's weather in Wuhan, save it as wuhan_weather.txt


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json sc

In [59]:
!cat wuhan_weather.txt

Date: Tuesday
Location: Wuhan, Hubei, China
Weather: Mostly cloudy
Temperature: 80°F
Precipitation: 0%
Humidity: 50%
Wind: 8 mph

In [38]:
!cat Beijing_Weather_Report.txt

cat: Beijing_Weather_Report.txt: No such file or directory
